In [1]:
import numpy as np
import pandas

from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import csv

In [2]:
from uci_datasets import Dataset

In [3]:
data_name = "song"
data = Dataset(data_name)

song dataset, N=515345, d=90


In [4]:
X = data.x
Y = data.y
#X = data[:,:-1]
#Y = data[:, -1].reshape(-1,1)

print(X)
print(Y)

ind = np.arange(X.shape[0])
print(ind)
prop = 0.9
if data_name == "houseelectric":
    prop = 6/9
n = int(X.shape[0] * prop)
print(n)

[[  -4.274   -45.213     8.8426 ...   11.544   -60.559   -33.82  ]
 [   3.2853   58.694    59.176  ...   11.855  -125.04     58.212 ]
 [   1.1868   44.437   -13.814  ...    6.1035 -160.48    -27.82  ]
 ...
 [  -8.8914  -99.723   -87.765  ...    1.8654  194.71     17.532 ]
 [   4.484    38.583    18.79   ...    1.8031   43.602     2.8297]
 [   3.2604  -40.927    23.822  ...   13.129    92.2       1.3212]]
[[-0.14915 ]
 [-0.33147 ]
 [-0.33147 ]
 ...
 [-0.14915 ]
 [ 0.073991]
 [-0.23616 ]]
[     0      1      2 ... 515342 515343 515344]
463810


In [5]:
from gpmaniflow.models.BezierProcess2 import BezierProcess
from gpflow.likelihoods import Gaussian
from gpflow.models import SGPR
from gpflow.inducing_variables import InducingPoints
import gpflow

import tensorflow as tf
import tensorflow_probability as tfp

from gpflow.utilities import print_summary

2022-05-11 23:42:18.517769: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-11 23:42:18.517813: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
BezierRMSE = np.zeros(3)
BezierRMedSE = np.zeros(3)
BezierTestLL = np.zeros(3)


ITERATIONS = 10000
ORDER = 20
BATCH_SIZE = 500
INPUT_DIM = X.shape[1]

for i in range(3):
    print('Split', i + 1, 'of', 20)
    np.random.seed(666 + i) # The devil's seed
    np.random.shuffle(ind)
    
    ### GET TRAINING SPLIT AND PREPROCESS ###
    
    Xtrain = X[ind[:n], :]; Ytrain = Y[ind[:n], :]
    Xtest = X[ind[n:], :]; Ytest = Y[ind[n:], :]
    
    mi, ma = Xtrain.min(axis=0), Xtrain.max(axis=0)
    mami = ma - mi
    mami = np.where(mami > 1e-6, mami, 1.) # Don't divide by zero
    Xtrain = (Xtrain - mi) / mami
    Xtest = (Xtest - mi) / mami

    mi = 0.01
    ma = 0.99
    mami2 = np.where(mami > 1e-6, mami, 0.)
    Xtrain = Xtrain * (ma - mi) + mi
    Xtest = Xtest * (ma - mi) + mi

    meany, stdy = np.average(Ytrain), np.std(Ytrain)
    Ytrain = (Ytrain - meany) / stdy
    if data_name == "houseelectric":
        print('Standardising Ytest.')
        Ytest = (Ytest - meany) / stdy 
    ############## BEZIER GP ##################
    
    m = BezierProcess(input_dim = INPUT_DIM, orders = ORDER, likelihood = Gaussian(), num_data = Xtrain.shape[0])
    m.likelihood.variance.assign(1.0)
    gpflow.set_trainable(m.likelihood.variance, False)
    
    minibatch_size = BATCH_SIZE

    train_dataset = tf.data.Dataset.from_tensor_slices((Xtrain, Ytrain)).repeat().shuffle(Xtrain.shape[0])
    train_iter = iter(train_dataset.batch(minibatch_size))
    
    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.001)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)

    iterations = ITERATIONS
    print('Training....')
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
    
    gpflow.set_trainable(m.BN, False) # FIX VARIATIONAL PARAMS
    gpflow.set_trainable(m.likelihood.variance, True) # AND TRAIN ONLY LIKELIHOOD VARIANCE
    
    training_loss = m.training_loss_closure(train_iter, compile=True)
    optimizer = tf.optimizers.Adam(learning_rate = 0.01)

    @tf.function
    def optimization_step():
        optimizer.minimize(training_loss, m.trainable_variables)
    for step in range(iterations):
        optimization_step()
        if step % 500 == 0:
            elbo = -training_loss().numpy()
            print('Split:', i + 1, 'Step:', step, 'Elbo:', elbo)
    
    
    test_dataset = tf.data.Dataset.from_tensor_slices((Xtest, Ytest))
    test_iter = iter(test_dataset.batch(minibatch_size))
    print('Testing...')
    SE = 0.
    loglik = 0.
    for batch in test_iter:
        xb = batch[0]
        yb = batch[1]
        preds, predvar = m.predict_f(xb)
        SE += np.sum((yb - (meany + stdy*preds.numpy()))**2)
        
        pred_mean, pred_var = m.predict_y(xb)
        pred_dist = tfp.distributions.Normal(loc=meany + pred_mean*stdy, scale=stdy*pred_var**0.5)
        loglik += tf.reduce_sum(pred_dist.log_prob(yb)).numpy()
        
    RMSE = np.sqrt( SE / Xtest.shape[0] )
    loglik = loglik / Xtest.shape[0]
    print('RMSE:', RMSE)
    print('Test log-likelihood:', loglik)
    ### Append results
    BezierRMSE[i] = RMSE
    BezierRMedSE[i] = 0
    BezierTestLL[i] = loglik
    
print('Experiment concluded!')

Split 1 of 20


2022-05-11 23:42:21.646949: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-11 23:42:21.646992: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-11 23:42:21.647020: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martinj): /proc/driver/nvidia/version does not exist
2022-05-11 23:42:21.647386: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-11 23:42:21.887285: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but 

Training....


2022-05-11 23:42:40.963473: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-11 23:42:49.474455: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 151200000 exceeds 10% of free system memory.
2022-05-11 23:42:49.602296: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 151200000 exceeds 10% of free system memory.
2022-05-11 23:42:52.337044: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 151200000 exceeds 10% of free system memory.


Split: 1 Step: 0 Elbo: -661158.458903993
Split: 1 Step: 500 Elbo: -655883.2821833781
Split: 1 Step: 1000 Elbo: -631580.1401240951
Split: 1 Step: 1500 Elbo: -619275.3280258181
Split: 1 Step: 2000 Elbo: -620069.4470509918
Split: 1 Step: 2500 Elbo: -619252.0263961077
Split: 1 Step: 3000 Elbo: -635957.176649345
Split: 1 Step: 3500 Elbo: -626902.4074380873
Split: 1 Step: 4000 Elbo: -609164.7297457366
Split: 1 Step: 4500 Elbo: -609868.2765263155
Split: 1 Step: 5000 Elbo: -612334.164645659
Split: 1 Step: 5500 Elbo: -604588.1326895896
Split: 1 Step: 6000 Elbo: -619515.906871013
Split: 1 Step: 6500 Elbo: -606777.5941659523
Split: 1 Step: 7000 Elbo: -581417.8326436223
Split: 1 Step: 7500 Elbo: -602495.3174961233
Split: 1 Step: 8000 Elbo: -601881.3525115625
Split: 1 Step: 8500 Elbo: -593689.2647028702
Split: 1 Step: 9000 Elbo: -578012.8407454108
Split: 1 Step: 9500 Elbo: -587372.2112972975
Split: 1 Step: 0 Elbo: -579355.3633020656
Split: 1 Step: 500 Elbo: -564415.5497991972
Split: 1 Step: 1000 El

In [7]:
print('Iterations:', ITERATIONS)
print('Order:', ORDER)
print('Batch size:', BATCH_SIZE)
print('Average RMSE:', np.mean(BezierRMSE), 'Standard deviation:', np.std(BezierRMSE))
print('Average RMedSE:', np.mean(BezierRMedSE), 'Standard deviation:', np.std(BezierRMedSE))
print('Average test-ll:', np.mean(BezierTestLL), 'Standard deviation:', np.std(BezierTestLL))

Iterations: 10000
Order: 20
Batch size: 500
Average RMSE: 10274.212397784215 Standard deviation: 14529.24601590042
Average RMedSE: 0.0 Standard deviation: 0.0
Average test-ll: -0.6828724536233043 Standard deviation: 0.018733207547008125
